In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn import preprocessing
lr = preprocessing.LabelEncoder()

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# reading the csv file for input data
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('C:/Users/Ashima Singhal/Downloads/covid_19_data.csv')

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
#transforming the data according to our requirments
df1['Target'] = lr.fit_transform(df1['Target'])
df1.head()

In [ ]:

df1['County'].unique()

In [ ]:
df2.rename(columns={'Province/State': 'State'}, inplace=True)
df2.rename(columns={'Country/Region': 'Country'}, inplace=True)

In [ ]:
fig = px.choropleth(df2,
                   locations='Country',
                   locationmode = 'country names',
                   color = 'Confirmed',
                   hover_name = 'Country',
                   animation_frame = 'Last Update')
fig.update_layout(
                title_text = 'spread of coronavirus',
                title_x = 0.5,
                geo = dict(
                showframe = False,
                showcoastlines = False,))
fig.show()

In [ ]:
df_confirmed, df_fatalities = [x for _, x in df1.groupby(df1['Target']>0)]
df_confirmed

In [ ]:
fig = px.choropleth(df_confirmed,
                   locations='County',
                   locationmode = 'country names',
                   color = 'TargetValue',
                   hover_name = 'County',
                   animation_frame = 'Date')
fig.update_layout(
                title_text = 'spread of coronavirus',
                title_x = 0.5,
                geo = dict(
                showframe = False,
                showcoastlines = False,))
fig.show()

In [ ]:
dataframe = df2.groupby('ObservationDate')[['Recovered','Confirmed','Deaths']].sum()

In [ ]:
fig1 = go.Bar(
         
          x = dataframe.index,
          y = dataframe.Recovered,
          name = 'Recovered')
fig2 = go.Bar(
        
          x = dataframe.index,
          y = dataframe.Confirmed,
          name = 'Confirmed')
fig3 = go.Bar(
          
          x = dataframe.index,
          y = dataframe.Deaths,
          name = 'Deaths')
dt = [fig1, fig2, fig3]
layout = go.Layout(title='Latest Stats')
figure = go.Figure(data = dt, layout = layout)
figure.show()

In [ ]:
from numpy import NaN, NAN , nan
df2['Country'].value_counts(dropna = False).head()
#no nan values

In [ ]:
df2.head()

In [ ]:
df_confirmed = pd.read_csv('C:/Users/Ashima Singhal/Downloads/time_series_covid_19_confirmed.csv')
df_confirmed.rename(columns={'Province/State':'State', 'Country/Region': 'Country'}, inplace=True)
df_confirmed.head(10)

In [ ]:
df_confirmed = df_confirmed[["State","Lat","Long","Country"]]
df_temp = df2.copy()
df_temp['Country'].replace({'Mainland China': 'China'}, inplace=True)
df_latlong = pd.merge(df_temp, df_confirmed, on=["Country", "State"])
df_latlong

In [ ]:
fig = px.density_mapbox(df_latlong, 
                        lat="Lat", 
                        lon="Long", 
                        hover_name="State", 
                        hover_data=["Confirmed","Deaths","Recovered"], 
                        animation_frame="ObservationDate",
                        color_continuous_scale="Portland",
                        radius=7, 
                        zoom=0,height=700)
fig.update_layout(title='Worldwide Corona Virus Cases Time Lapse - Confirmed, Deaths, Recovered',
                  font=dict(family="Courier New, monospace",
                            size=18,
                            color="#7f7f7f")
                 )
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=0)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

In [ ]:
dataframe.head(10)

In [ ]:
y = np.array(dataframe.iloc[:,1])
y

In [ ]:
train_dataset_confirmed = pd.DataFrame()
train_dataset_confirmed['ds'] = pd.to_datetime(dataframe.index)
train_dataset_confirmed['y']  = y
train_dataset_confirmed

In [ ]:
prophet_basic = Prophet()
prophet_basic.fit(train_dataset_confirmed)

In [ ]:
future = prophet_basic.make_future_dataframe(periods = 500)
future.tail(2)

In [ ]:
forecast = prophet_basic.predict(future)

In [ ]:
basic_plot = prophet_basic.plot(forecast)

In [ ]:
fig1 = prophet_basic.plot_components(forecast)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = prophet_basic.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), prophet_basic, forecast)

In [ ]:
pro_change = Prophet(changepoint_range=0.9)
forecast = pro_change.fit(train_dataset_confirmed).predict(future)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)

In [ ]:
pro_change= Prophet(n_changepoints=20, yearly_seasonality=True, changepoint_prior_scale=0.08)
forecast = pro_change.fit(train_dataset_confirmed).predict(future)
fig= pro_change.plot(forecast);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast)